# Procesamiento de Grafos / Redes Sociales: Kevin Bacon
[https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#Bacon_numbers](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#Bacon_numbers)

You are given a network of actors, where there is an edge between two actors if they have played together in some movie between 1995 and 2004

## Una primer mirada a los data sets que vamos a utilizar

Tenemos dos data sets:

- **imdb_actors_key_noheader.tsv:** Informacion sobre los actores (id, nombre, cantidad de peliculas actuadas, generos por peliculas, etc).
- **imdb_actor_edges.tsv:** Network propiamente dicha (relaciones de actor a actor por id y cantidad de veces que actuaron juntos). 

In [1]:
# carga de raw data
# tsv: tab separated value
actors = sc.textFile('data/imdb_actors_key_noheader.tsv', 8)
print('Cantidad de Actores:')
print(actors.count())

Cantidad de Actores:
17577


In [4]:
# realizas splits de las tuplas por \t (tab)
actors.map(lambda x: tuple(x.split('\t')))\
      .take(3)

[(u'15629',
  u'"Rudder, Michael (I)"',
  u'12',
  u'Thriller',
  u'Action:1,Comedy:1,Drama:1,Fantasy:1,Horror:1,NULL:2,Romance:1,Sci-Fi:1,Thriller:2,War:1'),
 (u'5026',
  u'"Morgan, Debbi"',
  u'16',
  u'Drama',
  u'Comedy:2,Documentary:1,Drama:6,Horror:2,NULL:3,Romance:2'),
 (u'11252',
  u'"Bellows, Gil"',
  u'33',
  u'Drama',
  u'Comedy:6,Documentary:1,Drama:7,Family:1,Fantasy:1,Horror:1,Mystery:2,NULL:2,Romance:6,Short:2,Thriller:4')]

In [5]:
# la red tiene nodos de la forma (actor1, actor2, veces_que_actuaron_juntos)
network = sc.textFile('data/imdb_actor_edges.tsv')
print 'Cantidad de Aristas/Edges:'
print network.count()

network.map(lambda x: tuple(x.split('\t')))\
      .take(3)

Cantidad de Aristas/Edges:
287074


[(u'17776', u'17778', u'6'), (u'5578', u'9770', u'3'), (u'5578', u'929', u'2')]

## Inicializando los RDDs

Vamos a inicializar los RDDs que utilizaremos

**Nota** Dado que estaremos usando estos RDDs en muchos casos de forma iterativa, realizaremos la operacion ``.cache()`` sobre los mismos, asegurandonos de que esa forma una vez que es calculado al haberse aplicado una accion cada nodo mantenga en memoria la particion que calculo.

Esto es importante al realizar algoritmos iterativos, dado que permite hasta una mejora de velocidad de 10x.

Para mas referencia: [https://spark.apache.org/docs/latest/programming-guide.html#rdd-persistence](https://spark.apache.org/docs/latest/programming-guide.html#rdd-persistence)

In [6]:
# inicializamos algunos rdds que vamos a utilizar en conjunto con nuestro network

# de la informacion de actores nos quedamos con su identificados y nombre
actors_key = actors.map(lambda x: tuple(x.split('\t')))\
                    .map(lambda x:(int(x[0]),x[1]))\
                    .cache()

actors_key_cat = actors.map(lambda x: tuple(x.split('\t')))\
                    .map(lambda x:(int(x[0]),(x[1],x[3])))\
                    .cache()

In [7]:
# actors_key_cat tiene nodos de la forma (actor_id, nombre_actor)
print("Ejemplo de actores de actors_key:")
print actors_key.take(5)

Ejemplo de actores de actors_key:
[(15629, u'"Rudder, Michael (I)"'), (5026, u'"Morgan, Debbi"'), (11252, u'"Bellows, Gil"'), (5150, u'"Dray, Albert"'), (4057, u'"Daly, Shane (I)"')]


In [8]:
# actors_key_cat tiene nodos de la forma (actor_id, (nombre_actor, nombre_categoria_mas_frecuente))
print("Ejemplo de actores de actors_key_cat:")
print actors_key_cat.take(5)

Ejemplo de actores de actors_key_cat:
[(15629, (u'"Rudder, Michael (I)"', u'Thriller')), (5026, (u'"Morgan, Debbi"', u'Drama')), (11252, (u'"Bellows, Gil"', u'Drama')), (5150, (u'"Dray, Albert"', u'Comedy')), (4057, (u'"Daly, Shane (I)"', u'Drama'))]


In [9]:
# Verificamos que podamos realizar algo con todos esto :)
KEVIN_BACON_ID=3257
kevin = actors_key_cat.filter(lambda x:x[0]==KEVIN_BACON_ID)
print "Nuestro Heroe:"
print kevin.collect()

Nuestro Heroe:
[(3257, (u'"Bacon, Kevin"', u'Drama'))]


In [10]:
# inicializamos el RDD de la network
network = sc.textFile('data/imdb_actor_edges.tsv')
print network.count()
network = network.map(lambda x:tuple(x.split('\t')))\
                 .map(lambda x:(int(x[0]),int(x[1]),int(x[2])))\
                 .cache()

287074


In [11]:
# network tiene nodos de la forma (actor1, actor2, cantidad_de_veces_que_actuaron_juntos)
print "Ejemplo de 5 tuplas del network RDD:"
print network.take(5)

Ejemplo de 5 tuplas del network RDD:
[(17776, 17778, 6), (5578, 9770, 3), (5578, 929, 2), (5578, 9982, 2), (1835, 6278, 2)]


## Estructuras para un Grafo

Las representacion posibles que tenemos son:

- **Lista de Aristas**
    - Una tupla por arista del tipo (nodo1, nodo2).
    - Puede incluir algun otro tipo de informacion relevante para la relacion.
- **Lista de Adyacencias**
    - Una tupla por nodo del tipo (nodo, [lista de vecinos])
    
    
Inicialmente trabajaremos con el grafo como una lista de aristas, luego para atacar otros problemas trabajaremos en spark con una lista de adyacencias.

Sin embargo podemos ver alguna informacion interesante respecto al grafo con el que estaremos trabajando.

In [12]:
print 'Cantidad Nodos (Actores): ' + str(actors_key.count())
print 'Cantidad de Aristas (Conexiones): ' + str(network.count())

Cantidad Nodos (Actores): 17577
Cantidad de Aristas (Conexiones): 287074


## Analizando Algunas propiedades de los Datos

In [13]:
# cuales son los generos posibles
actors_key_cat.map(lambda x: x[1][1])\
              .distinct()\
              .collect()

[u'Mystery',
 u'Romance',
 u'Drama',
 u'Action',
 u'',
 u'Fantasy',
 u'Horror',
 u'Comedy',
 u'Animation',
 u'Music',
 u'Sci-Fi',
 u'Adventure',
 u'War',
 u'Adult',
 u'Musical',
 u'Western',
 u'Thriller',
 u'Family',
 u'Crime']

In [14]:
# cuantas personas tenemos por generos posibles
actors_per_usual_category = actors_key_cat.map(lambda x: (x[1][1],1))\
                                          .reduceByKey(lambda x,y: x+y)\
                                          .collect();

for actor in actors_per_usual_category:
    print str(actor[0]) + ' ' + str(actor[1])

Mystery 66
Romance 1027
Drama 7302
Action 537
 68
Fantasy 137
Horror 171
Comedy 2928
Animation 284
Music 631
Sci-Fi 421
Adventure 63
War 69
Adult 1811
Musical 21
Western 24
Thriller 947
Family 359
Crime 711


## 10 actores que trabajaron con a mayor cantidad de personas

In [15]:
mn = network.flatMap(lambda x:[(x[0],1),(x[1],1)])
mn = mn.reduceByKey(lambda x,y:x+y)
mn = mn.join(actors_key_cat)
#mn = mn.filter(lambda x: x[1][1][1] != 'Adult')

In [19]:
top_actors = mn.takeOrdered(10,lambda x:-x[1][0])
for actor in top_actors:
    print actor[1][1][0]+'('+str(actor[1][0]) + ' personas) Genero:' + str(actor[1][1][1])

"Davis, Mark (V)"(784 personas) Genero:Adult
"Sanders, Alex (I)"(610 personas) Genero:Adult
"North, Peter (I)"(599 personas) Genero:Adult
"Marcus, Mr."(584 personas) Genero:Adult
"Tedeschi, Tony"(561 personas) Genero:Adult
"Dough, Jon"(555 personas) Genero:Adult
"Stone, Lee (II)"(545 personas) Genero:Adult
"Voyeur, Vince"(533 personas) Genero:Adult
"Lawrence, Joel (II)"(500 personas) Genero:Adult
"Steele, Lexington"(493 personas) Genero:Adult


In [20]:
# Que pasa con Kevin Bacon?
bm = mn.filter(lambda x:x[0]==KEVIN_BACON_ID).take(1)
print "Kevin Bacon trabajo con " + str(bm[0][1][0]) + " personas"

Kevin Bacon trabajo con 101 personas


# One Degree of Kevin Bacon

Queremos conocer las personas que se encuentra a un grado de separacion de Kevin Bacon

In [21]:
# filtramos aquellos aristas que tengan a kevin bacon
kevin1 = network.filter(lambda x:x[0]==KEVIN_BACON_ID or x[1]==KEVIN_BACON_ID)
# emitimos aquellos actor_id de los actores que trabajaron con Kevin
kevin1 = kevin1.map(lambda x:x[0] if x[1]==KEVIN_BACON_ID else x[1])
print kevin1.collect()
print "Actors a 1 grado de separacion de Kevin Bacon: " + str(kevin1.count())

[8087, 6895, 708, 8982, 10966, 2662, 7105, 519, 7762, 8578, 3169, 8996, 8999, 9386, 5612, 3160, 4248, 3242, 3133, 1064, 3143, 9079, 2896, 3308, 3305, 3306, 481, 3884, 701, 8334, 6131, 555, 6745, 15285, 3243, 3267, 3292, 1459, 3266, 6502, 3279, 3142, 3295, 3317, 569, 7566, 756, 2918, 6900, 3930, 3361, 3983, 565, 713, 3253, 3252, 7884, 3287, 6716, 9377, 3154, 9511, 2917, 1495, 9388, 10877, 3520, 689, 9237, 3923, 3176, 16485, 3869, 3255, 3183, 9409, 1627, 9449, 726, 716, 9406, 5744, 10659, 3213, 7020, 6492, 8468, 2926, 3309, 3235, 11464, 3721, 1253, 8751, 3870, 1047, 724, 2664, 3354, 3239, 5725]
Actors a 1 grado de separacion de Kevin Bacon: 101


In [22]:
#obtenemos los nombres de los actores a un grado de separacion.
kevin1_names = kevin1.map(lambda x:(x,1))\
                     .join(actors_key)\
                     .map(lambda x:x[1][1])

bacon1_actors = kevin1_names.collect()
for actor in bacon1_actors:
    print actor

"Danes, Claire"
"Wilson, Luke (I)"
"Taylor, Elizabeth (I)"
"Hoffman, Dustin"
"Harris, Ed (I)"
"Pollak, Kevin"
"Danson, Ted"
"Sinise, Gary"
"Hanks, Tom"
"Bisset, Jacqueline"
"Ice-T"
"Harden, Marcia Gay"
"Grant, Hugh (I)"
"Bonham Carter, Helena"
"Jackson, Janet (I)"
"Ford, Harrison (I)"
"Shue, Elisabeth"
"Jolie, Angelina"
"Gere, Richard"
"Moore, Demi"
"Keaton, Michael"
"Fishburne, Laurence"
"Quinlan, Kathleen"
"Penn, Sean"
"Robbins, Tim (I)"
"Dern, Laura"
"Leigh, Jennifer Jason"
"Cher (I)"
"Goldberg, Whoopi"
"Crudup, Billy"
"Ryan, Meg"
"Lauper, Cyndi"
"Grunberg, Greg"
"Dunn, Kevin (I)"
"Spears, Britney"
"Roberts, Julia"
"Wahlberg, Mark (I)"
"Lange, Jessica"
"Smith Jr., Eddie Bo"
"Parker, Sarah Jessica"
"Howard, Clint"
"Fonda, Bridget"
"Paxton, Bill"
"Reitman, Ivan"
"Dillon, Matt (I)"
"Spacey, Kevin"
"Lane, Diane (I)"
"Travolta, John"
"Norwood, Brandy"
"Gibson, Mel (I)"
"Brolin, Josh"
"Riegert, Peter"
"Crawford, Thomas (I)"
"Matlin, Marlee"
"Helgeland, Brian"
"Allen, Karen (I)"
"Madonna (

## Grado Promedio de la Red

In [23]:
total_deg = network.flatMap(lambda x:[(x[0],1),(x[1],1)])\
                   .reduceByKey(lambda x,y: x+y)

num_actors = total_deg.count()

print "Numero total de actores: " + str(num_actors)

Numero total de actores: 17577


In [24]:
acum_deg = total_deg.map(lambda x:x[1]).reduce(lambda x,y: x+y)

print "Grado Promedio: " + str((float(acum_deg)/num_actors)/2)

Grado Promedio: 16.3323661603


## N-Degrees of Kevin Bacon

Para poder realizar el calculo de los N-Degrees de Kevin Bacon (conocidos como Bacon Numbers) realizaremos una implementacion distribuida de BFS.

Para poder realizar la implementacion de N-Degrees of Kevin Bacon vamos a necesitar convertirlo en lista de adyacencia, 

```
(A,B)(A,C)(B,C) a una representacion del tipo (A,[B,C]),(B,[C]).
```

Pero en particular para implementar algoritmos de camino minimos con un formato particular.

```
(node_id, (distance, status, [list of neighbors], trace))
```

En el caso particular para calcular los Bacon Numbers vamos a usar la siguiente representacion:

```
[(node_id,(distance_from_kevin,status,[neighbors], trace)),....(node,(distance_from_kevin,status,[neighbors], trace))]
```

### Convirtiendo el grafo en lista de adyacencia

In [25]:
import itertools

# First we convert each (A,B) into (A,B) and (B,A)
adjlist = network.flatMap(lambda x:[(x[0],x[1]),(x[1],x[0])])

# veamos especificamente que pasa con Kevin Bacon
# uso de set para evitar duplicados, etc.
kadj = adjlist.groupByKey()\
             .filter(lambda x:x[0]==KEVIN_BACON_ID)\
             .map(lambda x:(x[0],(set(x[1]))))

kadj_driver = kadj.take(1)

print kadj_driver

print 'Cantidad de vecinos de Kevin Bacon: ' + str(len(kadj_driver[0][1]))


[(3257, set([519, 9237, 1047, 1064, 555, 8751, 565, 569, 6716, 3133, 3142, 3143, 7762, 3160, 6745, 1627, 5725, 16485, 3169, 2918, 2662, 3176, 3183, 5744, 2664, 10877, 3721, 3213, 8334, 4248, 9377, 3235, 3239, 9386, 3243, 9388, 689, 3252, 3253, 3255, 701, 9406, 9409, 3266, 3267, 708, 11464, 713, 716, 3279, 724, 10966, 3287, 3292, 3295, 7884, 1253, 9449, 3306, 8999, 3308, 3309, 6895, 6900, 3317, 726, 8468, 8982, 3354, 3869, 3870, 3361, 8996, 9511, 3884, 2896, 3923, 3930, 6492, 2917, 6502, 7020, 2926, 9079, 8578, 7566, 3983, 8087, 10659, 1459, 15285, 756, 3305, 3520, 7105, 1495, 481, 5612, 3154, 6131, 3242]))]
Cantidad de vecinos de Kevin Bacon: 101


### Llevando a la representacion necesaria para calcular los Bacon Numbers 

In [ ]:
# Usando groupByKey tenemos toda la informacion para poder armar la lista de adyacencia
# usando un set oara eliminar los duplicados.
# Inicializamos la distancia como infinito con 999
# status en -
# la lista de vecinos de ese nodo
# trace en 0
adjlist = adjlist.groupByKey()\
                 .map(lambda x:(x[0],(999,'-',list(set(x[1])),0)))\
                 .cache()

total_nodes = adjlist.count()
print "Numero total de nodos en nuestro grafo: " + str(total_nodes)

In [ ]:
# Nuestro RDD ahora tiene la forma [(node,(distance_from_kevin,status,[neighbors], trace)),....(node,(distance_from_kevin,status,[neighbors], trace))]
print adjlist.take(2)

In [ ]:
# El siguiente paso es marcar todos los nodos que son vecinos de Kevin Bacon como: 
#    1. a distancia 1
#    2. y como procesados en el status con 'P'
# Para hacer esto tomamos el nodo de Kevin Bacon y generamos nuevos nodos para los vecinos de la forma
# (node_id,(1,'P',[])).
# Luego en la fase de reduceByKey vamos a hacer un join de todos los nodos en uno unico (esperemos!)

# Para hacer esto definimos la funcion proc_node como la funcion que procesa un nodo 
# y vamos a comenzar el procesamiento desde KEVIN_BACON_ID (Kevin Bacon)
def proc_node(node):
    # Recibimos un nodo de la forma (nodeid, distance_from_kevin, status, list_of_neighbors, trace)
    # Generamos nuevos nodos para cada vecino agregando 1 a la distancia del nodo actual (!)
    if node[1][0]==999:
        new_distance = 1
    else:
        new_distance = node[1][0]+1
    ret = []
    for neighbor in node[1][2]:
        ret.append((neighbor,(new_distance,'P',[],node[0])))
    # We also update the node as already processed!
    ret.append((node[0],(node[1][0],'DONE',node[1][2],node[1][3])))
    return ret

# Definimos la funcion de reduccion
def reduce_nodes(n1,n2):
    # Recibimos dos nodos de la forma (distance_from_kevin,status,list_of_neighbors,trace) 
    # No quedamos con la distancia minima
    # si uno de los nodos esta marcado como 'DONE' nos quedamos con 'DONE'
    # si uno de los nodos esta marcado como 'P' nos quedamos con 'P'
    # nos quedamos con la lista mas larga de vecinos (alguna podria estar vacia)
    # Mantenemos un trace de la minima distancia
    new_status = ''
    if n1[1]=='P' or n2[1]=='P':
        new_status = 'P'
    if n1[1]=='DONE' or n2[1]=='DONE':
        new_status = 'DONE'
    if n1[0]<n2[0]:
        new_distance = n1[0]
        trace = n1[3]
    else:
        new_distance = n2[0]
        trace = n2[3]
    new_list = n1[2]
    if len(n2[2])>len(n1[2]):
        new_list = n2[2]
    return (new_distance,new_status,new_list,trace)

### Primera Pasada

Procesamos inicialmente solo el nodo de Kevin Bacon. Luego de esta Primera pasada deberiamos obtener solamente 101 con Bacon Numbers.

In [ ]:
# Primera Pasada: Nuestra primer tarea es procesar solo el nodo de Kevin Bacon (KEVIN_BACON_ID)
adjlist = adjlist.flatMap(lambda x:proc_node(x) if x[0]==KEVIN_BACON_ID else [x])
adjlist = adjlist.reduceByKey(reduce_nodes)

In [ ]:
# calculamos cuantos luego de la primer pasada tienen el bacon number
# estos serian los que estan a un grado de separacion
# y por lo que comprobamos anteriormente deberian ser 101
def bn_stats(adjlist,total_nodes):
    k_with_number = adjlist.filter(lambda x:x[1][0]<999)
    print "Numero de nodos con Bacon Number: " + str(k_with_number.count()) + "/" + str(total_nodes)

bn_stats(adjlist,total_nodes)

In [ ]:
def some_with_bacon_numbers(adjlist, degree): 
    print "Algunos Actores with Bacon Numbers"
    kb_1_names = adjlist.filter(lambda x:x[1][0]==degree)\
                         .map(lambda x:(x[0],1))\
                         .join(actors_key)\
                         .map(lambda x:x[1][1])\

    for name in kb_1_names.take(20):
        print name

some_with_bacon_numbers(adjlist,1)

### Segunda Pasada

Nuestro segundo paso consiste en procesar todos aquellos nodos con status 'P'

In [ ]:
# Nuestro siguiente paso consiste en procesar todos los nodos con status 'P'
adjlist = adjlist.flatMap(lambda x:proc_node(x) if x[1][1]=='P' else [x])
adjlist = adjlist.reduceByKey(reduce_nodes)

bn_stats(adjlist,total_nodes)
some_with_bacon_numbers(adjlist,2)

### Siguientes N-Pasadas Iterativas

Seguimos procesando entonces los nodos que estan en status 'P' hasta que no queden mas por procesar o no puedan procesarse mas.

In [ ]:
for i in range(0,6):
    adjlist = adjlist.flatMap(lambda x:proc_node(x) if x[1][1]=='P' else [x])
    adjlist = adjlist.reduceByKey(reduce_nodes)
    bn_stats(adjlist, total_nodes)

### Algunas verificaciones notables

In [ ]:
# Francella, Guillermo (5222)
print adjlist.filter(lambda x:x[0]==5222).join(actors_key_cat).map(lambda x: (x[0],x[1][1][0],x[1][0][0])).collect()
# Luis Brandoni (5223)
print adjlist.filter(lambda x:x[0]==5224).join(actors_key_cat).map(lambda x: (x[0],x[1][1][0],x[1][0][0])).collect()
# Gaston Pauls (4763)
print adjlist.filter(lambda x:x[0]==4763).join(actors_key_cat).map(lambda x: (x[0],x[1][1][0],x[1][0][0])).collect()

### Resumen: Caminos minimos desde un nodo

- **Usamos una estructura de nodo de lista de adyacencia con algunas variables extras para ayudar al procesamiento.**
    - (node_id, (distance, status, [list of neighbors]))
- **El estado inicial es solo con el nodo inicio en status ‘P’ y distancia 0.**
- **El resto de los nodos status ‘-’ y distancia Infinita (999) con nodos que vamos a tener que procesar.**
- El algoritmo es iterativo, por lo que en cada iteracion hacemos:
    - Por cada nodo marcado como 'P' procesamos sus vecinos
    - Por cada vecino generamos un nuevo nodo de la forma (node_id, (distance+1,'P',[]))
    - Luego en la fase de reduce nos quedaremos la distancia mínima y la lista de vecinos (alguno de los nodos la tiene)

## Random Walks

Concepto a ser desarrollado mas adelante en la materia, en particular para comprender PageRank (Teletransportacion!).

**Idea Base: Comenzar en uno o varios nodos al azar y en cada paso movernos a un nodo vecino de forma aleatoria.**

Los random walks tienen muchísimas aplicaciones.

- Determinar la centralidad (importancia) de nodos o aristas
- Encontrar nodos parecidos a otros
- Detectar clusters / comunidades
- Sistemas de Recomendaciones

### En Spark

Comenzar con un conjunto de nodos al azar.

- Por cada nodo elegir un vecino al azar y agregarlo a los resultados.
- Repetir este proceso “n” veces (longitud de los Random Walks)
- Observación: La cantidad de random walks es constante pero cada random walk tiene un nodo nuevo en cada iteración

### Centralidad (Importancia) de Nodos Utilizando Random Walks

In [ ]:
# Centrality by Random Walks!
# Select some random nodes from the graph
# From those nodes start a random walk
# Then just compute 1 for each node we have found!
import numpy as np

# llevamos la representacion que usamos para los bacon numbers 
# a una tipica representacion de lista de adyacencia
graph = network.flatMap(lambda x:[(x[0],x[1]),(x[1],x[0])])\
               .groupByKey()\
               .map(lambda x: (x[0], list(set(x[1]))))\
               .cache()
graph.take(1)

In [ ]:
# Como primer paso tomamos una muestra aleatoria con reemplazo
my_sample = graph.sample(True,0.3)

# todos los nodos seleccionados son parte de los random walks
all_the_nodes = my_sample.map(lambda x:(x[0],1))

print "Tomamos una muestra de: "+ str(my_sample.count()) + " nodos"

# Esta funcion toma un nodo y retorna un nuevo nodo seleccionando un random neighbor
# De cada nodo en la muestra seleccionar un random neighbor
def pick_random_neighbor(node):
    neighbor = np.random.choice(node[1])
    return (neighbor,'node')

for i in range(0,50):
    # Seleccionar un random neighbor de cada nodo
    my_sample = my_sample.map(lambda x:pick_random_neighbor(x)).cache()
    # hace un join con grafo para obtener una lista de vecinos
    my_sample = my_sample.join(graph).map(lambda x:(x[0],x[1][1])).cache()
    # adicionar lo nuevos nodos a la lista de nodos visitados
    just_nodes = my_sample.map(lambda x:(x[0],1))
    all_the_nodes = all_the_nodes.union(just_nodes).cache()
    print "Round: "+str(i)+" Total number of nodes:"+str(all_the_nodes.count())
print "----------------------------------------------------------------------"    
all_the_nodes = all_the_nodes.reduceByKey(lambda x,y:x+y)
nodes_with_names = all_the_nodes.join(actors_key_cat)
#print nodes_with_names.take(5)
central_actors =  nodes_with_names.takeOrdered(50,lambda x:-x[1][0])
i=0
for actor in central_actors:
    i=i+1
    print str(i)+". "+actor[1][1][0]+" ("+actor[1][1][1]+") = "+str(actor[1][0])